In [1]:
import numpy as np
import pandas as pd
import yfinance as yfin
import matplotlib.pyplot as plt
from datetime import datetime
from sdocks.consts import InvestmentData
from sdocks.performance import PerfMetrics
from sdocks.indicators import TechnicalIndicator
from plotly import graph_objects as go
from pandas_datareader import data as web

In [2]:
yfin.pdr_override()

In [3]:
# data=pd.read_csv('AWU.csv')
data=web.DataReader(
    # 'META',
    'BRK-B',
    start='2015-12-31',end=str(datetime.now())[:10]
).reset_index()

eod_data=data[['Date','Close']]
eod_data.columns=['date','close']
dates=[str(v)[:10] for v in list(eod_data.date.values)]
prices=list(eod_data.close.values)

[*********************100%%**********************]  1 of 1 completed


In [4]:
investment_data=InvestmentData(
    dates, 
    prices,
    'BRK-B',
    list(data.High.values),
    list(data.Low.values),
    list(data.Volume.values)
)
ti=TechnicalIndicator(investment_data)

In [5]:
op=ti.macd_signal()

In [6]:
fig=go.Figure()
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(op['dates']),
        y=op['signal'],
        name='MACD Signal'
    )
)

In [7]:
op=ti.bollinger_signal()

In [9]:
fig=go.Figure()
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(op['dates']),
        y=op['signal'],
        name='Bollinger Signal'
    )
)

In [8]:
chaikin_money_flow=ti.chaikinMoneyFlow()
fig=go.Figure()
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(chaikin_money_flow['dates']),
        y=chaikin_money_flow['chaikin_money_flow'],
        name='Chaikin Money Flow'
    )
)
fig.add_trace(
    go.Scatter(
        x=[pd.to_datetime(chaikin_money_flow['dates'][0]),pd.to_datetime(chaikin_money_flow['dates'][-1])],
        y=[0,0],
        name='reference line'
    )
)

In [ ]:
MACD=ti.macd()

In [10]:
bollinger=ti.bollingerBand()

In [ ]:
fig=go.Figure()
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(MACD['dates']),
        y=MACD['values'],
        name='MACD of META'
    )
)
fig.add_trace(
    go.Scatter(
        x=[pd.to_datetime(MACD['dates'][0]),pd.to_datetime(MACD['dates'][-1])],
        y=[0,0],
        name='reference line'
    )
)
fig.show()

In [11]:
short_sma=ti.sma(5)
long_sma=ti.sma(20)

In [12]:
fig=go.Figure()
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(investment_data.dates),
        y=investment_data.prices,
        name='price'
    )
)
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(short_sma['dates']),
        y=short_sma['values'],
        name='MA(5)'
    )
)
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(long_sma['dates']),
        y=long_sma['values'],
        name='MA(20)'
    )
)

# Add the upper bound with fill to the main line
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(bollinger['dates']), 
        y=bollinger['upper'], 
        fill=None, 
        mode='lines', 
        line_color='orange', 
        name='Bollinger Upper Bound'
    )
)

# Add the main line
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(bollinger['dates']), 
        y=bollinger['mid'], 
        fill='tonexty', 
        mode='lines',
        line_color='orange', 
        name='Bollinger Main'
    )
)

# Add the lower bound with fill to the main line
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(bollinger['dates']), 
        y=bollinger['lower'], 
        fill='tonexty', 
        mode='lines', 
        line_color='red', 
        name='Bollinger Lower Bound'
    )
)
fig.show()

In [ ]:
investment_data=InvestmentData(dates, prices)
investment_data.get_return()
investment_data.get_log_return()
data=investment_data.get_data()

In [ ]:
benchmark_data=InvestmentData(
    [str(v)[:10] for v in benchmark_data['Date'].values], 
    list(benchmark_data['Close'].values)
)
benchmark_data.get_log_return()
benchmark_data=benchmark_data.get_data()

In [ ]:
data=investment_data.get_data()
data=data[data.date >= '2016-01-01']
perf=PerfMetrics(
    data,
    0,
    benchmark_log_return=benchmark_data.dropna()['log return'].values,
    drawdown_method='percent',
    calmer_years=10
)
payload=perf.get_payload()

In [ ]:
payload

In [ ]:
fig=go.Figure()
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(data.date.values),
        y=data.price.values,
        name='prices'
    )
)
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(
            [payload['max_drawdown']['trough_date'], payload['max_drawdown']['peak_date']]
        ),
        y=[
            payload['max_drawdown']['trough_price'], payload['max_drawdown']['peak_price']
        ],
        name=f"Max Drawdown: {payload['max_drawdown']['max_drawdown']*100:.2f} %"
    )
)
fig.update_layout(
    dict(
        title='AWU'
    )
)